In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 4.9 MB/s eta 0:00:00


In [2]:
import torch
import numpy as np
import re
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load MetRex dataset
dataset = load_dataset("scale-lab/MetRex")

# Function to extract combinational depth from delay column
def extract_depth_from_delay(delay_text):
    match = re.search(r"The critical path goes through the following gates: (.*?). Delay of", delay_text)
    if match:
        gates = match.group(1).split(", ")
        return len(gates)  # Number of gates in the critical path
    return None

# Apply extraction
dataset = dataset["train"].map(lambda x: {"combinational_depth": extract_depth_from_delay(x["delay"])})

# Filter out samples where depth couldn't be extracted
dataset = dataset.filter(lambda x: x["combinational_depth"] is not None)

# Convert to LLM fine-tuning format
def format_example(example):
    return {
        "input_text": f"Predict the combinational depth of this RTL code:\n\n{example['RTL']}",
        "target_text": example["combinational_depth"],
    }

dataset = dataset.map(format_example)

# Split dataset into train/test
dataset = dataset.train_test_split(test_size=0.1, seed=42)

# Load CodeBERT tokenizer & model
model_name = "microsoft/codebert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize inputs
def tokenize_function(example):
    return tokenizer(example["input_text"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Convert labels to float for regression
def convert_labels(example):
    example["labels"] = float(example["target_text"])
    return example

tokenized_dataset = tokenized_dataset.map(convert_labels)

# Load model with regression configuration
config = AutoConfig.from_pretrained(model_name, num_labels=1, problem_type="regression")
model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)

# Freeze all layers except classifier
for param in model.roberta.parameters():
    param.requires_grad = False
model.roberta.eval()

# Training settings
training_args = TrainingArguments(
    output_dir="./checkpoints",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    logging_steps=10,
    learning_rate=1e-4,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none",
    push_to_hub=False,
    fp16=torch.cuda.is_available(),
)

# MSE Loss for regression
def compute_metrics(pred):
    preds = pred.predictions.squeeze()
    labels = pred.label_ids.squeeze()
    return {"mse": np.mean((preds - labels) ** 2)}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

# Train model
trainer.train()

# Save final model
trainer.save_model("./final_model")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/963 [00:00<?, ?B/s]

metrex.json:   0%|          | 0.00/137M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25868 [00:00<?, ? examples/s]

Map:   0%|          | 0/25868 [00:00<?, ? examples/s]

...

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss,Mse
1,20.509600,645.684265,645.684265
2,28.647400,638.376526,638.376526
3,47.443300,639.260254,639.260193


Test Accuracy: 16.54%


In [4]:
import pandas as pd

In [6]:
# Evaluate and save predictions
predictions = trainer.predict(tokenized_dataset["test"])
predictions_float = predictions.predictions.squeeze()
predictions_int = np.round(predictions_float).astype(int)
np.savetxt("test_outputs.txt", predictions_int, fmt="%d")

# Create final dataset with RTL, true labels, predicted output (float and rounded)
final_data = pd.DataFrame({
    "RTL": dataset["test"]["RTL"],
    "True_Label": dataset["test"]["target_text"],
    "Predicted_Float": predictions_float,
    "Predicted_Rounded": predictions_int
})
final_data.to_csv("final_predictions.csv", index=False)

# Calculate accuracy
true_labels = tokenized_dataset["test"]["labels"]
accuracy = accuracy_score(true_labels, predictions_int)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 16.54%


In [19]:
# Convert dataset to pandas DataFrame
df_train = pd.DataFrame(dataset["train"])
# Save as CSV
df_train.to_csv("cleaned_data_train.csv", index=False)
df_test = pd.DataFrame(dataset["test"])
df_test.to_csv("cleaned_data_test.csv", index=False)